<a href="https://colab.research.google.com/github/cerr/pycerr-notebooks/blob/main/autosegment_CT_Lung_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the OARs on a lung CT scan using pyCERR.  

## AI model
* The segmentation model was trained and validated on CT scans used for RT planning. It does not work optimally on diagnostic CTs.
* The trained model is distributed along with python libraries and other dependencies via a conda package.
* The model requires acess to a GPU.

### Running the model

* The Conda environment containing model weights and dependencies is downloaded to: condaEnvPath: `/content/pretrainedMode/'`
* Inference script location: `os.path.join(condaEnvPath,'CT_LungOAR_incrMRRN/model_wrapper/run_inference_nii.py')`

```python
python run_inference_nii.py <input_nii_directory> <output_nii_directory>
```

# Install pyCERR

pyCERR is used for data import/export and transformation.

In [0]:
%%capture
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR"

# Download pretrained segmentation model



In [0]:
%%capture
# Location of LungOAR model
modelDir ='/content/pretrainedModel'

# Download packaged environment for the AI model
boxLink = 'https://mskcc.box.com/shared/static/6ycs5rytze54i8ec9gd2dz8e52s2i5vv.gz'
saveFileName = 'ct_lung_oar.tar.gz'

!mkdir -p {modelDir}
!wget {boxLink} -O {saveFileName}
!tar xf {saveFileName} -C {modelDir}
!rm {saveFileName}

# Functions for data pre- and post-processing

## Identify the input scan and resize slices to 512x512

In [0]:
import cerr
from cerr.dataclasses import structure
from cerr.contour import rasterseg as rs
from cerr.utils import identifyScan, imageProc

def processInputData(planC):

  # Identify scan index in  planC
  scanIdS = {"imageType": "CT SCAN"}
  matchScanV = identifyScan.getScanNumForIdentifier(scanIdS, planC,
                                                    False)

  # Extract scan
  scanNum = matchScanV[0]
  scan3M = planC.scan[scanNum].getScanArray()
  mask3M = np.empty((0, 0, 0, 0))

  # Resize scan and import to planC
  inputImgSizeV = np.shape(scan3M)
  gridS = planC.scan[scanNum].getScanXYZVals()
  outputImgSizeV = [512, 512, inputImgSizeV[2]]
  method = 'padorcrop3d'
  procScan3M, __, resizeGridS = imageProc.resizeScanAndMask(scan3M,
                                mask3M, gridS, outputImgSizeV, method)

  return procScan3M, resizeGridS

## Import and refine AI segmentations

In [0]:
# Map output labels to structure names

strToLabelMap = {1:"Lung_Left", 2:"Lung_Right", 3:"Heart", 4:"Esophagus", \
                 5:"Cord", 6:"PBT"}
numLabel = len(strToLabelMap)

In [0]:
#Import label map to CERR
import glob

def postProcAndImportSeg(outputDir,procScanNum,scanNum,planC):
  niiGlob = glob.glob(os.path.join(outputDir,'*.nii.gz'))

  print('Importing ' + niiGlob[0]+'...')
  numStrOrig = len(planC.structure)
  planC = pc.load_nii_structure(niiGlob[0], procScanNum, planC, \
                              labels_dict = strToLabelMap)
  cpyStrNumV = np.arange(numStrOrig,len(planC.structure))
  numComponents = 1
  for label in range(numLabel):
    # Copy to original scan
    planC = structure.copyToScan(cpyStrNumV[label], scanNum, planC)
    origStr = len(planC.structure)-1
    strName =  strToLabelMap[label+1]
    # Post-process and replace input structure in planC
    procMask3M = imageProc.getLargestConnComps(origStr, numComponents,
                                                planC, saveFlag=True,
                                                replaceFlag=False,
                                                procSructName=strName)
  return planC

# Segment OARs

## Define I/O paths

Specify paths to the DICOM input data, desired output directory, and temporary (session) directory used to store intermediate results.

In [0]:
import os
from cerr import plan_container as pc

#Paths to input data and conda env with pre-trained models
inputDcmPath = '/content/sampleData/'  # Replace with path to dataset
outputDcmPath = '/content/AIoutput/'
sessionPath = '/content/temp'
condaEnvPath = '/content/pretrainedModel/'

if not os.path.exists(outputDcmPath):
  os.mkdir(outputDcmPath)

# Path to packaged conda environment
wrapperPath = os.path.join(condaEnvPath,'CT_LungOAR_incrMRRN', \
                          'model_wrapper','run_inference_nii.py')
activateScript = condaEnvPath+'/bin/activate'

## Run AI model

In [0]:
%%capture
import subprocess
import numpy as np
import cerr
from cerr import plan_container as pc
from cerr.ai import createSessionDir as cdir
from cerr.dcm_export import rtstruct_iod

# Loop over pyCERR files
fileList = os.listdir(inputDcmPath)
numFiles = len(fileList)
modality = 'CT'
scanNum = 0

for iFile in range(numFiles):

    dcmDir = os.path.join(inputDcmPath,fileList[iFile])

    # Create session dir to store temporary data
    modInputPath, modOutputPath = cdir.createSessionDir(sessionPath,
                                                        inputDcmPath)

    # Import DICOM scan to planC
    planC = pc.load_dcm_dir(dcmDir)

    # Pre-process data
    procScan3M, resizeGridS = processInputData(planC)
    planC = pc.import_scan_array(procScan3M, resizeGridS[0], \
            resizeGridS[1], resizeGridS[2], modality, scanNum, planC)
    procScanNum = len(planC.scan) - 1

    # Export inputs to NIfTI
    scanFilename = os.path.join(modInputPath,
                                f"{fileList[iFile]}_scan_3D.nii.gz")
    planC.scan[procScanNum].save_nii(scanFilename)

    # Apply model
    subprocess.run(f"source {activateScript} && python {wrapperPath} \
                  {modInputPath} {modOutputPath}", \
                  capture_output=False,shell=True,executable="/bin/bash")

    # Import results to planC
    planC = postProcAndImportSeg(modOutputPath,procScanNum,scanNum,planC)

    # Export segmentations to DICOM
    structFileName = fileList[iFile]+'_AI_seg.nii'
    structFilePath = os.path.join(outputDcmPath,structFileName)
    structNumV = list(np.arange(len(planC.structure)-numLabel,\
                                len(planC.structure)))
    seriesDescription = "AI Generated"
    rtstruct_iod.create(structNumV,structFilePath,planC,seriesDescription)


# Display results

## Display using matplotlib

In [0]:
from cerr.viewer import showMplNb

showMplNb(scanNum, structNumV, planC,\
          windowCenter=-400, windowWidth=2000)